# Semelhança de frase com NLU usando embeddings de BERT

Código Colab com mais exemplos (em inglês): https://colab.research.google.com/drive/1LtOdtXtRJ3_N8kYywPd5k2AJMCGcgAdN?usp=sharing


Vamos ver aqui como calcular similaridade entre sentenças usando *word embeddings* gerados pelo BERT, com a biblioteca *NLU* da *John Snow Labs*.

Vamos primeiro gerar os *embeddings* das sentenças e depois comparar cada par de frases em um conjunto de dados, para encontrar a frase mais semelhante a uma nova frase.

Obs: Este notebook foi executado no Colab

### Instalação de NLU e Java

In [1]:
import os
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu  pyspark==2.4.7 > /dev/null   

import nlu

### Download do nosso dataset 

Vamos usar esse conjunto de dados no nosso experimento, que contém títulos de TALK TEDs em português:

https://www.kaggle.com/miguelcorraljr/ted-ultimate-dataset

Se está usando Colab, baise o arquivo "ted_talks_pt-br.csv" e faça *upload* pelo Drive.


In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
# Apenas se está no Colab
# Montando Google Drive nesta instância
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
!cp -r "/content/drive/MyDrive/Cursos - recursos/ted_talks_pt-br.csv"  .


In [64]:
!ls

drive  jokes.csv  sample_data  ted_talks_pt-br.csv


In [65]:
df = pd.read_csv("ted_talks_pt-br.csv")

max_r = 5000
df = df.iloc[0:max_r]
df

,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript
0,1,Al Gore sobre como driblar a crise climática,Al Gore,{0: 'Al Gore'},{0: ['climate advocate']},{0: 'Nobel Laureate Al Gore focused the world’...,3523375,2006-02-25,2006-06-27,TED2006,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",272.0,977,"['alternative energy', 'cars', 'climate change...","{243: 'New thinking on the climate crisis', 54...",https://www.ted.com/talks/al_gore_averting_the...,Com o mesmo humor e humanidade que mostrou em ...,"Muito obrigado, Chris. É realmente uma grande ..."
1,7,"David Pogue diz que ""Simplicidade Vende.""",David Pogue,{0: 'David Pogue'},{0: ['technology columnist']},{0: 'David Pogue is the personal technology co...,1920837,2006-02-24,2006-06-27,TED2006,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",124.0,1286,"['computers', 'entertainment', 'interface desi...","{1725: '10 top time-saving tech tips', 2274: '...",https://www.ted.com/talks/david_pogue_simplici...,O colunista do New York Times David Pogue crit...,"Olá, Correio de Voz, velho amigo. (risos) Eu l..."
2,53,"Um conto de renovação urbana, por Majora Carter",Majora Carter,{0: 'Majora Carter'},{0: ['activist for environmental justice']},{0: 'Majora Carter redefined the field of envi...,2664009,2006-02-26,2006-06-27,TED2006,en,"['ar', 'bg', 'bn', 'ca', 'cs', 'de', 'en', 'es...",219.0,1116,"['MacArthur grant', 'activism', 'business', 'c...",{1041: '3 stories of local eco-entrepreneurshi...,https://www.ted.com/talks/majora_carter_greeni...,"Em um discurso emocionado, a ativista Majora C...",Se você está aqui hoje — e estou muito feliz p...
3,66,Será que as escolas matam a criatividade?,Sir Ken Robinson,{0: 'Sir Ken Robinson'},"{0: ['author', 'educator']}","{0: ""Creativity expert Sir Ken Robinson challe...",65051909,2006-02-25,2006-06-27,TED2006,en,"['af', 'ar', 'az', 'be', 'bg', 'bn', 'ca', 'cs...",4931.0,1164,"['children', 'creativity', 'culture', 'dance',...","{865: 'Bring on the learning revolution!', 173...",https://www.ted.com/talks/sir_ken_robinson_do_...,Sir Ken Robinson defende de maneira divertida ...,"Bom dia. Como estão? Tem sido ótimo, não tem? ..."
4,92,Hans Rosling mostra as melhores estatísticas q...,Hans Rosling,{0: 'Hans Rosling'},{0: ['global health expert; data visionary']},"{0: 'In Hans Rosling’s hands, data sings. Glob...",14501661,2006-02-22,2006-06-27,TED2006,en,"['ar', 'az', 'bg', 'bn', 'bs', 'cs', 'da', 'de...",628.0,1190,"['Africa', 'Asia', 'Google', 'demo', 'economic...","{2056: ""Own your body's data"", 2296: 'A visual...",https://www.ted.com/talks/hans_rosling_the_bes...,Você nunca viu dados apresentados desta forma....,"Há uns 10 anos, assumi a tarefa de ensinar des..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,60405,Uma perspectiva evolucionária sobre a saúde e ...,Lara Durgavich,{0: 'Lara Durgavich'},{0: ['biological anthropologist']},{0: 'Lara Durgavich brings historical context ...,604197,2019-03-10,2020-04-20,TEDxTufts,en,"['ar', 'en', 'pt-br']",9.0,927,"['biology', 'evolution', 'medicine', 'science'...",{2779: 'The biology of our best and worst selv...,https://www.ted.com/talks/lara_durgavich_an_ev...,"Como sua herança genética, cultura e história ...",Quando estava com quase nove semanas de gravid...
3897,62628,Como mudar sua mentalidade e escolher seu futuro,Tom Rivett-Carnac,{0: 'Tom Rivett-Carnac'},{0: ['political strategist']},{0: 'A political strategist focused on creatin...,771460,2020-04-22,2020-04-22,TED2020: The Prequel,en,"['en', 'pt-br']",31.0,954,"['climate change', 'choice', 'leadership', 'so...",{2480: 'The inside story of the Paris climate ...,https://www.ted.com/talks/tom_rivett_carnac_ho...,"Quando se trata de grandes problemas da vida, ...",Eu nunca pensei que daria minha palestra TED n...
3898,62707,O ciclo de vida de um par de t

### Gerando os *embeddings*

Vamos gerar os *embeddings* para as frases do corpus, usando a coluna *Title*.

Aqui vamos usar o modelo multilingual, `xx.embed_sentence`.

Obs: Até o momento de criação deste notebook, não existe modelo para português como `pt.embed_sentence.bert_large_cased`.

In [44]:
pipe = nlu.load('xx.embed_sentence') # modelo multilingual


sent_bert_multi_cased download started this may take some time.
Approximate size to download 638.6 MB
[OK!]


Vamos gerar a matriz com os *embeddings* de cada sentença, usando a coluna "title".

In [66]:
predictions = pipe.predict(df.title, output_level='document')
predictions

,title,document,text,xx_embed_sentence_embeddings
origin_index,,,,
0,Al Gore sobre como driblar a crise climática,Al Gore sobre como driblar a crise climática,Al Gore sobre como driblar a crise climática,"[-0.3526768684387207, -0.2555399239063263, 0.5..."
1,"David Pogue diz que ""Simplicidade Vende.""","David Pogue diz que ""Simplicidade Vende.""","David Pogue diz que ""Simplicidade Vende.""","[0.03163643181324005, -0.008944074623286724, 0..."
2,"Um conto de renovação urbana, por Majora Carter","Um conto de renovação urbana, por Majora Carter","Um conto de renovação urbana, por Majora Carter","[-0.1523960679769516, 0.18735936284065247, 0.0..."
3,Será que as escolas matam a criatividade?,Será que as escolas matam a criatividade?,Será que as escolas matam a criatividade?,"[-0.0009366358863189816, 0.2620916962623596, 0..."
4,Hans Rosling mostra as melhores estatísticas q...,Hans Rosling mostra as melhores estatísticas q...,Hans Rosling mostra as melhores estatísticas q...,"[-0.36017054319381714, -0.5067304968833923, 0...."
...,...,...,...,...
3896,Uma perspectiva evolucionária sobre a saúde e ...,Uma perspectiva evolucionária sobre a saúde e ...,Uma perspectiva evolucionária sobre a saúde e ...,"[-0.27822327613830566, -0.23190081119537354, 1..."
3897,Como mudar sua mentalidade e escolher seu futuro,Como mudar sua mentalidade e escolher seu futuro,Como mudar sua mentalidade e escolher seu futuro,"[0.052607957273721695, 0.14145636558532715, 0...."
3898,O ciclo de vida de um par de tênis - Angel Chang,O ciclo de vida de um par de tênis - Angel Chang,O ciclo de vida de um par de tênis - Angel Chang,"[0.26456764340400696, 0.4353902041912079, 0.31..."


### Encontrando N sentenças mais similares no dataset dada uma nova sentença

Sentenças com distâncias pequenas entre seus *embeddings* serão mais similares entre si. 

In [70]:
## coluna do dataset com os embeddings
e_col = 'xx_embed_sentence_embeddings'

# Esta função calcula as distâncias de uma frase em previsões [sent_iloc] para todas as outras frases usando a incorporação definida por e_col
def get_sim_df_for_iloc(sent_iloc, predictions=predictions,e_col=e_col, pipe=pipe):
  embed_mat = np.array([x for x in predictions[e_col]])
  # calcula a distância entre cada par de incorporação
  sim_mat = cosine_similarity(embed_mat,embed_mat)
  print("Similaridades para sentença : " + df.iloc[sent_iloc].title)
  # adiciona os scores ao dataframe
  df['sim_score'] = sim_mat[sent_iloc]
  return df 

# indice da sentenca a ser comparada no dataframe
sentence_to_compare=98

sim_df_for_one_sent = get_sim_df_for_iloc(sentence_to_compare,predictions,e_col)
sim_df_for_one_sent.sort_values('sim_score', ascending = False)

Similaridades para sentença : Natalie MacMaster toca violino em tempo "reel" (ritmo folclórico escocês)


,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript,sim_score
98,117,"Natalie MacMaster toca violino em tempo ""reel""...",Natalie MacMaster,"{0: 'Natalie MacMaster', 1: 'Thomas Dolby'}","{0: ['fiddler'], 1: ['electronic music pioneer']}",{0: 'Natalie MacMaster is a star of Cape Breto...,835898,2002-02-02,2007-05-01,TED2002,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",69.0,311,"['entertainment', 'history', 'music', 'perform...","{45: ""An 11-year-old's magical violin"", 296: '...",https://www.ted.com/talks/natalie_macmaster_ca...,A violinista Natalie MacMaster e o diretor mus...,♫ Como a urze ♫ ♫ na encosta ♫ ♫ como eles nos...,1.000000
371,447,MacMaster + Leahy tocam o violino,Natalie MacMaster,{0: 'Natalie MacMaster'},{0: ['fiddler']},{0: 'Natalie MacMaster is a star of Cape Breto...,1500199,2003-02-02,2009-01-30,TED2003,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",53.0,1127,"['collaboration', 'culture', 'entertainment', ...",{1298: 'There are no mistakes on the bandstand...,https://www.ted.com/talks/natalie_macmaster_fi...,Natalie MacMaster e seu parceiro musical Donne...,Natalie MacMaster: Eu simplesmente vou começar...,0.719696
837,1083,"Ahn Trio: Uma interpretação moderna com piano,...",Ahn Trio,{0: 'Ahn Trio'},{0: ['piano trio']},"{0: 'With cello, piano and violin, sisters Mar...",852141,2010-12-08,2011-02-25,TEDWomen 2010,en,"['ar', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es...",100.0,565,"['entertainment', 'live music', 'performance',...","{138: 'A string quartet plays ""Blue Room""', 17...",https://www.ted.com/talks/ahn_trio_a_modern_ta...,"As três irmãs Ahn (violoncelista Maria, a pian...",(Aplausos) (Música) (Aplausos) Angella Ahn: Ob...,0.715518
271,325,"Nellie McKay canta ""A canção do cachorro""",Nellie McKay,{0: 'Nellie McKay'},{0: ['singer/songwriter']},"{0: 'Nellie McKay sings, croons, raps, grooves...",797969,2008-02-02,2008-08-22,TED2008,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",51.0,213,"['animals', 'entertainment', 'live music', 'mu...","{296: '""Mother of Pearl,"" ""If I Had You""', 287...",https://www.ted.com/talks/nellie_mckay_the_dog...,"Nellie McKay, uma fã dos animais, canta um tri...","Vou dedicar esta canção ao Carmelo, que foi po...",0.697930
1167,1435,Reuben Margolin: Esculpindo ondas em madeira e...,Reuben Margolin,{0: 'Reuben Margolin'},{0: ['kinetic sculptor']},"{0: ""Reuben Margolin's moving sculptures combi...",681672,2012-03-01,2012-05-02,TED2012,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",69.0,538,"['art', 'culture', 'design', 'nature']","{1164: 'Taking imagination seriously', 534: 'A...",https://www.ted.com/talks/reuben_margolin_scul...,"Reuben Margolin é um escultor do movimento, cr...","Geralmente gosto de trabalhar em meu atelier, ...",0.696964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,2873,Quem pertence à cidade?,OluTimehin Adegbeye,{0: 'OluTimehin Adegbeye'},"{0: ['writer', 'activist']}","{0: 'Writing on urban development, sexual and ...",2460018,2017-08-27,2017-09-08,TEDGlobal 2017,en,"['af', 'ar', 'cs', 'de', 'el', 'en', 'es', 'fa...",25.0,723,"['activism', 'Africa', 'government', 'cities',...","{2880: ""Living sculptures that stand for histo...",https://www.ted.com/talks/olutimehin_adegbeye_...,"Quando governos criam megacidades, os pobres n...",Cidades são como irmãs em uma grande família p...,0.315728
1641,1943,"Nós podemos ""ter tudo""?",Anne-Marie Slaughter,{0: 'Anne-Marie Slaughter'},{0: ['public policy thinker']},{0: 'Anne-Marie Slaughter has exploded the con...,1994599,2013-06-11,2014-03-12,TEDGlobal 2013,en,"['ar', 'cs', 'de', 'el', 'en', 'es', 'fr', 'he...",210.0,1031,"['inequality', 'men', 'policy', 'women', 'work...","{1040: 'Why we have too few women leaders', 19...",https://www.ted.com/talks/anne_marie_slaughter...,"Especialista em polí

In [71]:
# indice da sentenca a ser comparada no dataframe
sentence_to_compare=1757

sim_df_for_one_sent = get_sim_df_for_iloc(sentence_to_compare,predictions,e_col)
sim_df_for_one_sent.sort_values('sim_score', ascending = False)

Similaridades para sentença : Por que o universo existe?


,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript,sim_score
1757,2072,Por que o universo existe?,Jim Holt,{0: 'Jim Holt'},{0: ['writer and philosopher']},{0: 'Why is there something rather than nothin...,7865404,2014-03-13,2014-09-02,TED2014,en,"['ar', 'bg', 'bs', 'ca', 'cs', 'da', 'de', 'el...",565.0,1037,['philosophy'],"{2045: 'How do you explain consciousness?', 13...",https://www.ted.com/talks/jim_holt_why_does_th...,Por que existe algo em vez de nada? Em outras ...,Por que o universo existe? Por que existe... C...,1.000000
1766,2069,Qual é a próxima janela para o universo?,Andrew Connolly,{0: 'Andrew Connolly'},{0: ['astronomer']},{0: 'Andrew Connolly is helping to build the L...,1262944,2014-03-19,2014-09-16,TED2014,en,"['ar', 'de', 'el', 'en', 'es', 'fa', 'fr', 'he...",62.0,1059,"['astronomy', 'space', 'technology', 'universe...","{900: 'A 3D atlas of the universe', 1936: 'Wha...",https://www.ted.com/talks/andrew_connolly_what...,"Big data está em toda parte, até nos céus. Em ...","Em 1781, um compositor inglês, tecnólogo e ast...",0.813853
3755,23716,Quantos universos existem?,Chris Anderson,{0: 'Chris Anderson'},{0: ['head of ted']},{0: 'After a long career in journalism and pub...,1331445,2012-03-25,2020-01-16,TED-Ed,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",NaN,271,"['education', 'TED-Ed', 'space', 'animation', ...",{24392: 'Could human civilization spread acros...,https://www.ted.com/talks/chris_anderson_how_m...,O fato de ninguém saber a resposta desta pergu...,"(Música) Às vezes, quando estou em um longo vo...",0.811910
1752,2067,Podemos impedir o fim do mundo?,Martin Rees,{0: 'Martin Rees'},{0: ['astrophysicist']},"{0: ""Lord Martin Rees, one of the world's most...",1289116,2014-03-13,2014-08-25,TED2014,en,"['ar', 'bg', 'ca', 'de', 'el', 'en', 'es', 'fa...",296.0,412,"['future', 'humanity', 'science']","{42: 'Is this our final century?', 1835: 'Can ...",https://www.ted.com/talks/martin_rees_can_we_p...,"Uma Terra pós-apocalíptica, sem humanos, parec...","Dez anos atrás, escrevi um livro que chamei de...",0.796982
3131,33799,O que é consciência?,Michael Graziano,{0: 'Michael Graziano'},NaN,NaN,640616,2019-02-11,2019-02-11,TED-Ed,en,"['ar', 'bs', 'de', 'en', 'es', 'fa', 'fr', 'he...",NaN,300,"['TED-Ed', 'consciousness', 'animation', 'brai...",{23117: 'The surprising link between stress an...,https://www.ted.com/talks/michael_s_a_graziano...,Veja a lição completa: https://ed.ted.com/less...,Aqui estão duas imagens de uma casa. Há uma di...,0.787060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,981,Salão de jogos na web de Ze Frank,Ze Frank,{0: 'Ze Frank'},{0: ['web humorist']},{0: 'Ze Frank has been involved in online come...,2179857,2010-07-09,2010-10-15,TEDGlobal 2010,en,"['ar', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es...",149.0,1080,"['Internet', 'comedy', 'gaming', 'humanity', '...","{87: 'Nerdcore comedy', 383: 'A story of mixed...",https://www.ted.com/talks/ze_frank_my_web_play...,"Na web, um novo ""Amigo"" pode estar a apenas um...",Toda apresentação precisa desse slide. (Risos)...,0.234141
3316,37382,"""East Virginia"" / ""John Brown's Dream""",Nora Brown,{0: 'Nora Brown'},{0: ['musician']},{0: 'Nora Brown sings ballads and plays tradit...,260968,2019-01-24,2019-03-29,TED Salon: Education Everywhere,en,"['ar', 'en', 'es', 'fa', 'fr', 'it', 'ko', 'ku...",3.0,551,"['music', 'live music', 'vocals', 'performance...",{391: 'Walk the earth ... my 17-year vow of si...,https://www.ted.com/talks/nora_brown_east_virg...,"Em um cenário fascinante, a musicista Nora Bro...","(Música: ""East Virginia"") (Banjo) ♪ Sou ♪ ♪ Da...",0.221613
1120,1385,A Barata Beatbox,Greg Gage,{0: 'Greg Gage'},{0: ['neuroscientist']},{0: 'TED Fellow Greg Gage helps kids investiga...,843548,2011-11-17,2012-03-12,TED-Ed,en,"['ar', 'az', 'de', '